# Getting Started

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
data = pd.read_csv("../input/chocolate-bar-ratings/flavors_of_cacao.csv")

In [ ]:
data

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(data.corr(), annot=True)
plt.show()

In [ ]:
data.drop(['REF', 'Review\nDate'], axis=1, inplace=True)

In [ ]:
data

# Preprocesing

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna(axis=0)

In [ ]:
data.dtypes

In [ ]:
data.columns = ['Company', 'SpecificOrigin', 'CocoaPercent', 'Location', 'Rating', 'BeanType', 'BroadOrigin']

In [ ]:
data

In [ ]:
def removePercents(data):
    return data.apply(lambda x: float(x.strip('%')) / 100)

In [ ]:
data['CocoaPercent'] = removePercents(data['CocoaPercent'])

In [ ]:
len(data['SpecificOrigin'].unique())

In [ ]:
categorical_features = ['Company', 'SpecificOrigin', 'Location', 'BeanType', 'BroadOrigin']

In [ ]:
def onehot_encode(data, columns):
    for column in columns:
        dummies = pd.get_dummies(data[column])
        data = pd.concat([data, dummies], axis=1)
        data.drop(column, axis=1, inplace=True)
    return data

In [ ]:
data = onehot_encode(data, categorical_features)

In [ ]:
y = data['Rating']
X = data.drop('Rating', axis=1)

In [ ]:
X

In [ ]:
scaler = MinMaxScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [ ]:
inputs = tf.keras.Input(shape=(1655,))
x = tf.keras.layers.Dense(16, activation='relu')(inputs)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(
    optimizer=optimizer,
    loss='mse'
)

In [ ]:
model.summary()

In [ ]:
epochs = 10
batch_size = 32

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

In [ ]:
plt.figure(figsize=(14, 10))

plt.plot(range(epochs), history.history['loss'], color='b')
plt.plot(range(epochs), history.history['val_loss'], color='r')

plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.show()

In [ ]:
np.argmin(history.history['val_loss'])

In [ ]:
model.evaluate(X_test, y_test)